In [ ]:
import numpy as np
import matplotlib.pyplot as plt

##-- Lorenz parameters
sigma = 10.0
beta = 8.0/3.0
rho = 28.0

def f(x):
    X, Y, Z = x
    return np.array([
        sigma * (Y - X),
        X * (rho - Z) - Y,
        X * Y - beta * Z
    ])

##-- Change to RK4
def rk2_step(x, dt):
    k1 = f(x)
    k2 = f(x + 0.5 * dt * k1)
    return x + dt * k2

##-- Integrate
x0 = np.array([5.0, 5.0, 5.0])
dt = 0.01
T = 200.0
nsteps = int(T / dt)

traj = np.zeros((nsteps+1, 3))
traj[0] = x0

for n in range(nsteps):
    traj[n+1] = rk2_step(traj[n], dt)

In [ ]:
##-- Plot
fig = plt.figure(figsize=(10,5))
plt.subplot(131)
plt.plot(traj[:,0], traj[:,1],lw=1, color='b')
plt.xlabel("X")
plt.ylabel("Y")

plt.subplot(132)
plt.plot(traj[:,1], traj[:,2],lw=1, color='r')
plt.xlabel("Y")
plt.ylabel("Z")

plt.subplot(133)
plt.plot(traj[:,0], traj[:,2],lw=1, color='g')
plt.xlabel("X")
plt.ylabel("Z")

plt.tight_layout()
plt.show()

In [ ]:
##-- Plot
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111, projection='3d')
ax.plot(traj[:,0], traj[:,1], traj[:,2], lw=1, color='b')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.tight_layout()
plt.show()


In [ ]:
##-- Delayed Embedding
fig = plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot( traj[:,0], traj[:,1], 'b', lw=1 )
plt.title('X-Y')

plt.subplot(122)
lag = 15
plt.plot( traj[:-lag,0], traj[lag:,0], 'r', lw=1 )
plt.title('Delayed coordinates')

plt.tight_layout()
plt.show()

In [ ]:
##-- Autocorrelation
x = traj[:,0]  # your x(t) series

##-- Normalize
x = x - np.mean(x)
x = x / np.std(x)

##-- Compute autocorrelation
acf = np.correlate(x, x, mode='full')
acf = acf[acf.size//2:] / np.max(acf)  ##-- Normalize, take positive lags

##-- Find first zero-crossing
lag_zero = np.where(acf < 0)[0][0]
tau = lag_zero * dt

print("Suggested delay  based on first zero-crossing: tau = %.4f, index = %d" % (tau, lag_zero))

# Plot autocorrelation
plt.figure(figsize=(6,3))
plt.plot(acf)
plt.xlabel("Lag index")
plt.xlim([0,lag_zero])
plt.ylabel("Autocorrelation")
plt.show()


In [ ]:
##-- Twin Trajectories
sigma = 10.0
beta = 8.0/3.0
rho = 28.0

def f(x):
    X, Y, Z = x
    return np.array([
        sigma * (Y - X),
        X * (rho - Z) - Y,
        X * Y - beta * Z
    ])

##-- Change to RK4
def rk2_step(x, dt):
    k1 = f(x)
    k2 = f(x + 0.5 * dt * k1)
    return x + dt * k2

##-- Reach Attractor First -- Use as IC: [17.32992667, 21.31162256, 35.57265303]
x0 = np.array([17.32992667, 21.31162256, 35.57265303])
dt = 0.001
T = 20.0
nsteps = int(T / dt)
tvec = np.arange(nsteps+1)*dt

ntraj = 50
eps = 0.0001
traj = np.zeros((ntraj, nsteps+1, 3))
traj[0,0,:] = x0
traj[1:,0,:] = x0
traj[1:,0,:] += np.random.random( (ntraj-1,3) )*eps

for n in range(nsteps):
    for t in range(ntraj):
        traj[t,n+1,:] = rk2_step(traj[t,n,:], dt)


In [ ]:
# plt.plot( tvec, traj[0,:,1], 'b', lw=1 )
# plt.plot( tvec, traj[1,:,1], 'r', lw=1 )
# plt.show()

diff = np.mean( np.sum( (traj[0,:,:] - traj[1:,:,:])**2.0, axis=2)**0.5, axis=0)

plt.plot(tvec, diff)

plt.plot( tvec, eps*np.exp(tvec*0.904), '--r' ) 

plt.yscale('log')
plt.show()